<a href="https://colab.research.google.com/github/Amruth2503/Agentic-AI/blob/master/bank_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain langchain-community chromadb sentence-transformers transformers


IMPORT STATEMENTS

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from transformers import pipeline

import os

SAVE DATA TO FILES

In [ ]:
os.makedirs("data", exist_ok=True)

docs = {

    "credit_card_terms.txt": """
    Annual fee:

Card issuers may charge a fee once a year in lieu of the service they provide. Depending on the scope of service, this may range from a few hundred to a few thousand. However, some service providers may waive the fee for the first year while others, like IDFC FIRST Bank, offer credit cards free for lifetime use.

Credit limit:

This is the total spending limit or credit line given to you by the card provider. Your income levels, credit score, repayment history and tenure with the bank are some of the factors that determine your credit limit. With a good repayment history, banks may periodically offer higher limits to you.

Annual percentage rate:

APR is the banking terminology for the annualised interest charged if you do not pay the outstanding dues of a billing cycle in full within the due date. The interest is charged monthly for every month on unpaid dues. To know the monthly interest charged, simply divide the APR by 12.

Credit score:

Your credit score is a three-digit number represented on a scale of up to 900. A good credit score, usually above 750, makes getting a new credit card easier and sanctions for higher credit limits.

Billing cycle:

This is the period between the closing of the previous and the next statement date. Most banks usually have a 30-day billing cycle, and you must remember the dates for your card to plan expenses and finances accordingly.

Due date:

This is the fixed monthly date when all outstanding payments to the bank, including EMIs, must be paid. Making the payment well before the due date is recommended to avoid late-payment penalties and high-interest charges on unpaid dues.

Grace period:

This is the period from the date of issue of your statement till the payment due date. The grace period may range from 14 to 21 days, varying from one service provider to another. The most significant advantage is that no interest is charged on your outstanding dues during this period. This is an important feature to remember when you make use of a credit card.

Interest rate:

Interest charges on a credit card can differ. The interest rate applied to an EMI will be different and lower than that for extending your payment. It will also vary for a cash advance or an over-limit charge. Before signing up for a card, check how different transactions and services are charged.
    """
}

# Save docs to files
for name, content in docs.items():
    with open(f"data/{name}", "w") as f:
        f.write(content.strip())

LOAD DATA

In [ ]:
def load_and_split_all_documents():
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    all_chunks = []

    for filename in os.listdir("data"):
        with open(f"data/{filename}", "r") as f:
            content = f.read()
        doc = Document(page_content=content, metadata={"source": filename})
        all_chunks.extend(splitter.split_documents([doc]))

    return all_chunks

chunks = load_and_split_all_documents()

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(documents=chunks, embedding=embedding_model, persist_directory="bank_db")
vectordb.persist()


In [ ]:
qa_pipeline = pipeline("text-generation", model="tiiuae/falcon-rw-1b", max_new_tokens=256)


Device set to use cpu


In [ ]:
def generate_answer(query):
    docs = vectordb.similarity_search(query, k=3)
    context = "\n\n".join([doc.page_content for doc in docs])

    prompt = f"""
You are a banking assistant. Use the following context to answer clearly:

Context:
{context}

Question:
{query}

Answer:"""

    return qa_pipeline(prompt)[0]['generated_text'].split("Answer:")[-1].strip()


GENERATE ANSWERS

In [ ]:
query = "What is APR?"
print("Query:", query)
print("Answer:", generate_answer(query))


Query: What is APR?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: Annual Percentage Rate
APR is the banking terminology for the annualised interest charged if you do not pay the outstanding dues of a billing cycle in full within the due date. The interest is charged monthly for every month on unpaid dues. To know the monthly interest charged, simply divide the APR by 12.

APR is the banking terminology for the annualised interest charged if you do not pay the outstanding dues of a billing cycle in full within the due date. The interest is charged monthly for every month on unpaid dues. To know the monthly interest charged, simply divide the APR by 12.

APR is the banking terminology for the annualised interest charged if you do not pay the outstanding dues of a billing cycle in full within the due date. The interest is charged monthly for every month on unpaid dues. To know the monthly interest charged, simply divide the APR by 12.

APR is the banking terminology for the annualised interest charged if you do not pay the outstanding dues of a 